#### Cria repositório de reports

In [1]:
from utils.enums import Datas
from utils.checkpoints import verifyPath

teste_size = 0
main_data = Datas.MOTION
path_reports = f"report_results/{Datas.HAR.value}/{main_data.value}_{teste_size}/"

split_path = path_reports.split("/")
partial_path = ""
for i, part in enumerate(split_path):
    partial_path += part + "/"
    verifyPath(partial_path)


# Treina Pretexto

#### Hiperparâmentos

In [ ]:
import math
import torch
import numpy as np
import pandas as pd
from models.cnn1d import CNN1d
from utils.enums import Datas, Sets, ModelTypes
from data_modules.pretext import HarDataModule as HarDataModulePretext
from transforms.har import rotation, flip, noise_addition, permutation, scaling, time_warp, negation

printQtd = 1
num_epoch = 100
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Optim
learning_rate = 0.02
step_size = 20
gamma = 0.5

#### Carrega Dados

In [ ]:
transforms = [rotation, flip, noise_addition, permutation, scaling, time_warp, negation]
data_module = HarDataModulePretext(batch_size=batch_size, main_data = main_data)
train_dl, train_ds = data_module.get_dataloader(set=Sets.TRAIN.value, shuffle=True, transforms=transforms)
test_dl, test_ds   = data_module.get_dataloader(set=Sets.TEST.value, shuffle=False, transforms=transforms)
num_classes = len(transforms)

#### Define Modelo

In [ ]:
model_p = CNN1d(data_label=main_data.value, num_classes=num_classes, require_grad=True, type=ModelTypes.PRETEXT.value)
print(model_p)

#### Treinamento

In [ ]:
# import lightning as L
# trainer = L.Trainer(
#     max_epochs=10,
#     accelerator='cpu',
#     log_every_n_steps=1        
# )
# trainer.fit(model=model, train_dataloaders=train_dl)

optimizer, lr_scheduler = model_p.configure_optimizers(step_size=step_size, gamma=gamma, learning_rate=learning_rate)
optimizer, lr_scheduler = optimizer[0], lr_scheduler[0]

train_errors = []
validation_errors = []
best_val_loss = 500
n_total_steps = len(train_dl)
for epoch in range(num_epoch):
    # Treinamento
    model_p.train()
    train_loss = 0
    for i, batch in enumerate(train_dl):
        loss = model_p.training_step(batch)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % math.floor(n_total_steps/printQtd) == 0:
            print (f'Epoch [{epoch+1:4d}/{num_epoch}], Step [{i+1:4d}/{n_total_steps}], Loss: {loss.item():.4f}', end= "" if n_total_steps/printQtd+i >= n_total_steps else "\n")

    lr_scheduler.step()

    # Validação
    model_p.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in test_dl:
            val_loss += model_p.validation_step(batch)
    
    val_loss /= len(test_dl)
    print(f' Validation Loss: {val_loss:.4f}')

    train_errors.append(train_loss/len(train_dl))
    validation_errors.append(val_loss.item())

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model_p.state_dict()  # Salva os parâmetros do modelo

model_p.load_state_dict(best_model)

#### Avaliação

In [ ]:
accTotal = 0
predicted_values = []
real_values = []
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(num_classes)]
    n_class_samples = [0 for i in range(num_classes)]
    n_each_class_samples = [0 for i in range(num_classes)]

    for data, labels in test_dl:
        outputs = model_p(data)

        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for pred, real in zip (predicted, labels):
            predicted_values.append(pred.item())
            real_values.append(real.item())

        for i in range(labels.shape[0]):
            label = labels[i]
            pred  = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
            n_each_class_samples[pred] += 1

    accTotal = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {accTotal} %')

    for i in range(num_classes):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {test_ds.getLabel(i)} ({n_class_correct[i]}/{n_class_samples[i]} | {n_each_class_samples[i]}): {acc} %')

#### Salva resultados

In [ ]:
model_p.save_backbone(accuracy=accTotal, batch_size=batch_size, num_epoch=num_epoch)

pred_reports = pd.DataFrame({
    Sets.REAL.value: real_values,
    Sets.PREDICTION.value : predicted_values
})
pred_reports.to_csv(f"{path_reports}/predictions_{ModelTypes.PRETEXT.value}.dat", sep=" ", index=False)

train_reports = pd.DataFrame({
    Sets.TRAIN.value : train_errors,
    Sets.VALIDATION.value : validation_errors
})
train_reports.to_csv(f"{path_reports}/errors_{ModelTypes.PRETEXT.value}.dat", sep=" ", index=False)

# Treina Downstream

#### Hiperparâmentos

In [15]:
import torch
import math
import numpy as np
import pandas as pd
from data_modules.har import HarDataModule as HarDataModuleDownstram
from utils.enums import Datas, Sets, ModelTypes
from models.cnn1d import CNN1d

printQtd = 1
isFreezing = True
batch_size = 10
num_epoch = 500
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Optim config
learning_rate_bb = 0.02
step_size_bb = 120
gamma_bb = 0.5

learning_rate_ds = 0.01
step_size_ds = 100
gamma_ds = 0.5

#### Carrega Dados

In [16]:
data_module = HarDataModuleDownstram(batch_size=64)
train_dl, train_ds = data_module.get_dataloader(set=Sets.TRAIN.value, shuffle=True)
test_dl, test_ds   = data_module.get_dataloader(set=Sets.TEST.value, shuffle=True)
num_classes = len(train_ds.labels)

#### Define Modelo

In [17]:
model = CNN1d(
    data_label=main_data.value, 
    num_classes=num_classes, 
    require_grad=isFreezing, 
    type=ModelTypes.DOWNSTREAM.value
)
print(model)

CNN1d(
  (criterion): CrossEntropyLoss()
  (backbone): Backbone(
    (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv1d(6, 12, kernel_size=(2,), stride=(1,))
    (conv2): Conv1d(12, 24, kernel_size=(2,), stride=(1,))
    (conv3): Conv1d(24, 48, kernel_size=(2,), stride=(1,))
  )
  (pred_head): ProjectionHead(
    (linear1): Linear(in_features=288, out_features=6, bias=True)
  )
)


#### Treinamento

In [18]:
# import lightning as L
# trainer = L.Trainer(
#     max_epochs=10,
#     accelerator='cpu',
#     log_every_n_steps=1        
# )
# trainer.fit(model=model, train_dataloaders=train_dl)

optimizer_backbone, lr_scheduler_backbone = model.configure_backbone_optimizers(step_size=step_size_bb, gamma=gamma_bb, learning_rate=learning_rate_bb)
optimizer_downstream, lr_scheduler_downstream = model.configure_head_optimizers(step_size=step_size_ds, gamma=gamma_ds, learning_rate=learning_rate_ds)

train_errors = []
validation_errors = []
best_val_loss = 500
n_total_steps = len(train_dl)
for epoch in range(num_epoch):
    # Treinamento
    model.train()
    train_loss = 0
    for i, batch in enumerate(train_dl):
        loss = model.training_step(batch)
        train_loss += loss.item()
        optimizer_backbone.zero_grad()
        optimizer_downstream.zero_grad()
        loss.backward()
        optimizer_backbone.step()
        optimizer_downstream.step()
        
        if (i+1) % math.floor(n_total_steps/printQtd) == 0:
            print (f'Epoch [{epoch+1:4d}/{num_epoch}], Step [{i+1:4d}/{n_total_steps}], Loss: {loss.item():.4f}', end= "" if n_total_steps/printQtd+i >= n_total_steps else "\n")

    lr_scheduler_backbone.step()
    lr_scheduler_downstream.step()

    # Validação
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in test_dl:
            val_loss += model.validation_step(batch)
    
    val_loss /= len(test_dl)
    print(f' Validation Loss: {val_loss:.4f}')

    train_errors.append(train_loss/len(train_dl))
    validation_errors.append(val_loss.item())

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model.state_dict()  # Salva os parâmetros do modelo

model.load_state_dict(best_model)

Epoch [   1/500], Step [   1/1], Loss: 1.8195

/home/sr_rosa/.local/lib/python3.10/site-packages/lightning/pytorch/core/module.py:436: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


 Validation Loss: 1.7819
Epoch [   2/500], Step [   1/1], Loss: 1.7952 Validation Loss: 1.7599
Epoch [   3/500], Step [   1/1], Loss: 1.7622 Validation Loss: 1.7307
Epoch [   4/500], Step [   1/1], Loss: 1.7260 Validation Loss: 1.7122
Epoch [   5/500], Step [   1/1], Loss: 1.7029 Validation Loss: 1.7006
Epoch [   6/500], Step [   1/1], Loss: 1.6862 Validation Loss: 1.6926
Epoch [   7/500], Step [   1/1], Loss: 1.6733 Validation Loss: 1.6873
Epoch [   8/500], Step [   1/1], Loss: 1.6636 Validation Loss: 1.6840
Epoch [   9/500], Step [   1/1], Loss: 1.6560 Validation Loss: 1.6820
Epoch [  10/500], Step [   1/1], Loss: 1.6503 Validation Loss: 1.6808
Epoch [  11/500], Step [   1/1], Loss: 1.6458 Validation Loss: 1.6801
Epoch [  12/500], Step [   1/1], Loss: 1.6420 Validation Loss: 1.6797
Epoch [  13/500], Step [   1/1], Loss: 1.6385 Validation Loss: 1.6794
Epoch [  14/500], Step [   1/1], Loss: 1.6355 Validation Loss: 1.6788
Epoch [  15/500], Step [   1/1], Loss: 1.6329 Validation Loss: 1.

<All keys matched successfully>

#### Avaliação

In [19]:
accTotal = 0
predicted_values = []
real_values = []
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(num_classes)]
    n_class_samples = [0 for i in range(num_classes)]
    n_each_class_samples = [0 for i in range(num_classes)]

    for data, labels in test_dl:
        outputs = model(data)

        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for pred, real in zip (predicted, labels):
            predicted_values.append(pred.item())
            real_values.append(real.item())

        for i in range(labels.shape[0]):
            label = labels[i]
            pred  = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
            n_each_class_samples[pred] += 1

    accTotal = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {accTotal} %')

    for i in range(num_classes):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {test_ds.getLabel(i)} ({n_class_correct[i]}/{n_class_samples[i]} | {n_each_class_samples[i]}): {acc} %')

Accuracy of the network: 45.833333333333336 %
Accuracy of Run (4/4 | 5): 100.0 %
Accuracy of Sit (0/4 | 0): 0.0 %
Accuracy of Stair-down (3/4 | 4): 75.0 %
Accuracy of Stair-up (0/4 | 0): 0.0 %
Accuracy of Stand (0/4 | 0): 0.0 %
Accuracy of Walk (4/4 | 15): 100.0 %


#### Salva Resultados

In [20]:
model.save_full_model(accuracy=accTotal, batch_size=batch_size, num_epoch=num_epoch)

pred_reports = pd.DataFrame({
    Sets.REAL.value: real_values,
    Sets.PREDICTION.value : predicted_values
})
pred_reports.to_csv(f"{path_reports}/predictions_{ModelTypes.DOWNSTREAM.value}.dat", sep=" ", index=False)

train_reports = pd.DataFrame({
    Sets.TRAIN.value : train_errors,
    Sets.VALIDATION.value : validation_errors
})
train_reports.to_csv(f"{path_reports}/errors_{ModelTypes.DOWNSTREAM.value}.dat", sep=" ", index=False)